In [4]:
from tensorflow.keras.models import model_from_json
import cv2
import numpy as np
model = model_from_json(open("fer.json", "r").read())
net = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt","res10_300x300_ssd_iter_140000.caffemodel")
Emotion_list=[]
# initialize camera stream

# loop over the frames from the video stream
import imutils
import tensorflow as tf

In [5]:
vid=cv2.VideoCapture(0)
while True:
    ret,frame = vid.read()
    frame = imutils.resize(frame, width=400)
    #Mean subtraction is used to help combat illumination changes in the input images in our dataset
    #view mean subtraction as a technique used to aid our Convolutional Neural Networks.

    #Before we even begin training our deep neural network,
    #we first compute the average pixel intensity across all images in the training set for each of the Red, Green, and Blue channels.
    ##1 is scale factor
   ##blob is the binary long object classification


    # grab the frame dimensions and convert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,(300, 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()
    #print(detections)
    faceBoxes = []
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out detections by confidence
        if confidence < 0.7:
            continue

        # compute the (x, y)-coordinates of the bounding box for the
        # object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #print(gray_frame)
        #print(box)
        frameHeight = frame.shape[0]
        frameWidth = frame.shape[1]
        x1 = int(detections[0, 0, i, 3] * frameWidth)
        y1 = int(detections[0, 0, i, 4] * frameHeight)
        x2 = int(detections[0, 0, i, 5] * frameWidth)
        y2 = int(detections[0, 0, i, 6] * frameHeight)
        faceBoxes.append([x1, y1, x2, y2])
        #faceBoxes=box
        #print(faceBoxes)
        #print(faceBoxes[0][2])
        roi = gray_frame[y1:y1+y2, x1:x1+x2]
        print(roi)
        roi_new = cv2.resize(roi, (48, 48))
        print(roi_new.shape)
        roi_new=roi_new.reshape(1,48,48,1)
        #roi_gray_frame = gray_frame[faceBoxes[0][2]:faceBoxes[0][2] + faceBoxes[0][3], faceBoxes[0][0]:faceBoxes[0][0] + faceBoxes[0][3]]
        #cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

        emotionlistforadding=['Angry','Disgusted','Fearful','Happy','Neutral','Sad','Surprised']
        #break
        
        new_model = tf.keras.models.load_model('NewEmotion.hp5')
#         emotion_model.load_weights('emotion_model.h5')
        
        #emotion_model.load_weights('')
        #new_model = tf.keras.models.load_model('fer.hp5')
        #model.load_weights('fer.h5')
        emotion_prediction = new_model.predict(roi_new)
        #print(emotion_prediction)
        maxindex = int(np.argmax(emotion_prediction))
        print(maxindex)
        labelemotion= "{}".format("Emotion : " + emotion_dict[maxindex])
        Emotion_list.append(labelemotion)


    # draw the bounding box of the face along with the associated
    # probability
        text = "{:.2f}%".format(confidence * 100)


        ##adding the text for showing emotion
        #it is the coordinates of the bottom-left corner of the text string in the image. The coordinates are represented as tuples of two values
        #Font scale factor that is multiplied by the font-specific base size.(0.8)
        #BGR, we pass a tuple.255 0 0
        #gives the type of the line to be used.
        #2 is thickness

        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
        cv2.putText(frame, labelemotion, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (0, 255, 255), 2, cv2.LINE_AA)
    #cv2.putText(frame, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)


# show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

# if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

[[ 25  24  26 ... 119 120 123]
 [ 23  23  25 ... 118 117 121]
 [ 22  25  27 ... 116 117 119]
 ...
 [ 14  14  15 ...  10  10  10]
 [ 15  15  16 ...  10  10  10]
 [ 15  15  15 ...  10  10  10]]
(48, 48)
3
[[12 12 11 ... 86 86 87]
 [12 12 11 ... 88 87 87]
 [12 12 12 ... 90 89 89]
 ...
 [10 10 10 ...  7  7  7]
 [10 10 10 ...  7  8  7]
 [10 10 10 ...  7  8  7]]
(48, 48)
5
[[ 16  16  16 ... 105 104 106]
 [ 16  16  17 ... 104 106 108]
 [ 16  16  15 ... 104 106 107]
 ...
 [ 12  12  13 ...  10  10  10]
 [ 13  13  13 ...   9   9  10]
 [ 13  13  13 ...   9   9  11]]
(48, 48)
3
[[ 25  25  26 ... 115 120 122]
 [ 23  23  25 ... 117 120 122]
 [ 23  23  24 ... 119 120 120]
 ...
 [ 13  14  14 ...   9   9   9]
 [ 14  13  14 ...   9   9  10]
 [ 14  13  14 ...  10   9  10]]
(48, 48)
6
[[104  81  58 ... 120 120 125]
 [ 73  54  39 ... 121 118 120]
 [ 45  35  33 ... 123 119 118]
 ...
 [ 12  14  14 ...  38  38  36]
 [ 13  14  14 ...  37  37  36]
 [ 13  14  14 ...  35  35  36]]
(48, 48)
2
[[ 38  34  32 ... 120

5
[[ 23  24  26 ... 102  97 100]
 [ 23  24  25 ... 103  97  99]
 [ 22  23  24 ...  99  92 100]
 ...
 [ 13  12  10 ...  46  43  41]
 [ 13  11   9 ...  44  42  41]
 [ 13  11   9 ...  42  41  40]]
(48, 48)
4
[[ 26  24  23 ... 124 124 121]
 [ 23  22  22 ... 122 119 117]
 [ 22  22  22 ... 119 119 121]
 ...
 [ 12  13  14 ...  32  32  31]
 [ 12  13  14 ...  30  30  29]
 [ 13  13  14 ...  27  26  26]]
(48, 48)
4
[[ 31  31  29 ... 123 131 125]
 [ 30  30  29 ... 121 130 128]
 [ 27  26  25 ... 117 125 129]
 ...
 [ 12  14  15 ...  32  32  32]
 [ 12  13  14 ...  29  29  30]
 [ 13  14  15 ...  28  28  27]]
(48, 48)
3
[[ 31  29  28 ... 125 126 120]
 [ 31  29  27 ... 122 119 117]
 [ 30  28  26 ... 122 122 119]
 ...
 [ 14  15  15 ...  30  29  29]
 [ 14  15  15 ...  28  28  26]
 [ 13  15  15 ...  26  26  24]]
(48, 48)
5
[[ 33  32  32 ... 126 123 126]
 [ 33  33  33 ... 124 119 121]
 [ 30  31  28 ... 119 116 117]
 ...
 [ 13  14  14 ...  29  28  28]
 [ 13  14  15 ...  26  26  26]
 [ 13  13  15 ...  25  24 

4
[[ 28  28  28 ... 122 123 127]
 [ 29  29  27 ... 121 118 122]
 [ 25  24  26 ... 121 118 123]
 ...
 [ 16  16  16 ...  44  44  44]
 [ 16  16  16 ...  44  44  44]
 [ 15  16  16 ...  42  43  44]]
(48, 48)
6
[[ 29  28  26 ... 126 127 126]
 [ 29  28  27 ... 126 125 124]
 [ 29  30  30 ... 127 123 122]
 ...
 [ 14  14  15 ...  43  43  43]
 [ 14  14  15 ...  42  42  42]
 [ 15  14  14 ...  42  42  42]]
(48, 48)
5
[[ 29  30  29 ... 120 119 122]
 [ 30  30  28 ... 123 126 125]
 [ 31  30  28 ... 125 128 127]
 ...
 [ 13  14  15 ...  42  43  42]
 [ 13  13  14 ...  40  42  41]
 [ 15  14  14 ...  40  42  41]]
(48, 48)
6
[[ 30  28  27 ... 119 122 125]
 [ 32  28  26 ... 126 126 127]
 [ 31  29  28 ... 126 126 125]
 ...
 [ 14  13  14 ...  42  42  43]
 [ 14  14  14 ...  41  41  43]
 [ 14  13  14 ...  41  40  41]]
(48, 48)
5
[[ 30  28  29 ... 121 126 126]
 [ 29  28  28 ... 125 127 127]
 [ 30  29  29 ... 126 126 125]
 ...
 [ 15  15  16 ...  43  43  43]
 [ 15  15  15 ...  42  43  42]
 [ 15  16  16 ...  41  42 